# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)

In [4]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

#### cleaning non-binary columns:

In [6]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [7]:
bad_samples = []
for i in df.index:
    if df.loc[i,'related'] == 2:
        bad_samples.append(i)

In [8]:
len(bad_samples)

188

In [9]:
good_samples = [i for i in df.index if i not in bad_samples ]

In [10]:
# checking:
len(good_samples)+len(bad_samples), df.shape[0]

(26216, 26216)

In [11]:
df = df.iloc[good_samples,]
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [12]:
# final check:
df.related.value_counts()

1    19906
0     6122
Name: related, dtype: int64

#### splitting  feature vs target:

In [13]:
features = ['message']

In [14]:
X = df[features]
X.head(3)

,message
0,Weather update - a cold front from Cuba that c...
1,Is the Hurricane over or is it not over
2,Looking for someone but no name


In [15]:
non_target_cols = ['id','message','original','genre'] 
non_target_cols

['id', 'message', 'original', 'genre']

In [16]:
target_cols = df.columns[~df.columns.isin(non_target_cols)]
target_cols

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [17]:
Y = df[target_cols]
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [19]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [20]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [22]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize) ),
    ('tfidf', TfidfTransformer() ),
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X.message,Y)

In [25]:
(X_train.shape , Y_train.shape)

((19521,), (19521, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [27]:
from sklearn.multioutput import MultiOutputClassifier

In [28]:
clf = pipeline.fit(X_train, Y_train)

In [29]:
Y_pred = pipeline.predict(X_test)

In [30]:
(Y_test.shape, Y_pred.shape)

((6507, 36), (6507, 36))

In [31]:
[type(Y_test), type(Y_pred)]

[pandas.core.frame.DataFrame, numpy.ndarray]

In [32]:
Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
Y_pred.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
import numpy as np

In [34]:
from sklearn.utils.multiclass import type_of_target

In [35]:
[type_of_target(Y_test), type_of_target(Y_pred)]

['multilabel-indicator', 'multilabel-indicator']

In [36]:
import inspect
inspect.signature(classification_report)

<Signature (y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2)>

In [37]:
print(classification_report(Y_test,Y_pred,target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.93      0.88      4989
               request       0.81      0.44      0.57      1099
                 offer       0.00      0.00      0.00        27
           aid_related       0.76      0.59      0.67      2722
          medical_help       0.61      0.08      0.14       503
      medical_products       0.87      0.12      0.22       314
     search_and_rescue       0.62      0.04      0.08       182
              security       0.50      0.01      0.02       103
              military       0.75      0.05      0.10       219
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.45      0.59       402
                  food       0.83      0.41      0.55       736
               shelter       0.81      0.31      0.44       582
              clothing       0.88      0.08      0.14        93
                 money       0.86      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [38]:
print(f1_score((Y_test), (Y_pred),average='macro'))

0.238372073938


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
# parameters in CountVectorizer
inspect.signature(CountVectorizer)

<Signature (input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)>

In [40]:
# parameters in TfidfTransformer
inspect.signature(TfidfTransformer)

<Signature (norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)>

In [41]:
# parameters in RandomForestClassifier
inspect.signature(RandomForestClassifier)

<Signature (n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)>

In [42]:
list(pipeline.get_params().keys())

['memory',
 'steps',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimat

In [43]:
parameters = {
    'vect__tokenizer': [tokenize,None], # for count vectorizer
    'tfidf__norm': ['l1','l2'], # for tfidf
    'clf__estimator__criterion': ['gini','entropy'] # for Random Forest
}

# Note: using 'clf__criterion' as syntax generates an error:
# ValueError: Invalid parameter criterion for estimator MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
# By doing cv.get_params().keys() one can see the above one is the right syntax

In [44]:
from sklearn.model_selection import GridSearchCV

In [45]:
inspect.signature(GridSearchCV)

<Signature (estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score='raise', return_train_score='warn')>

In [46]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [50]:
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__tokenizer': [<function tokenize at 0x7ff3b584d620>, None], 'tfidf__norm': ['l1', 'l2'], 'clf__estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [52]:
pd.DataFrame(cv.cv_results_)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__criterion,param_tfidf__norm,param_vect__tokenizer,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,40.699703,0.607647,4.511940,0.012145,gini,l1,<function tokenize at 0x7ff3b584d620>,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.244506,0.234209,0.233748,0.237488,0.004966,4,0.807361,0.807746,0.809129,0.808078,0.000759
1,34.009803,0.446326,1.729761,0.005515,gini,l1,None,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.219302,0.223913,0.223452,0.222222,0.002073,8,0.806439,0.807899,0.806516,0.806951,0.000671
2,40.676352,0.273635,4.528909,0.030027,gini,l2,<function tokenize at 0x7ff3b584d620>,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.241432,0.237898,0.241893,0.240408,0.001785,2,0.808130,0.815430,0.810281,0.811280,0.003063
3,33.932946,0.253492,1.732456,0.006930,gini,l2,None,"{'clf__estimator__criterion': 'gini', 'tfidf__...",0.230675,0.218841,0.222069,0.223861,0.004995,6,0.807515,0.814431,0.807438,0.809795,0.003278
4,41.968250,0.507910,4.154640,0.021125,entropy,l1,<function tokenize at 0x7ff3b584d620>,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.241893,0.235439,0.248041,0.241791,0.005145,1,0.807899,0.809282,0.803135,0.806772,0.002633
5,33.579221,0.285388,1.312371,0.006536,entropy,l1,None,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.226218,0.221915,0.232826,0.226986,0.004488,5,0.795374,0.799754,0.797065,0.797398,0.001804
6,41.209486,0.533459,4.163903,0.042368,entropy,l2,<function tokenize at 0x7ff3b584d620>,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.243430,0.228062,0.245274,0.238922,0.007716,3,0.811741,0.816736,0.804057,0.810845,0.005215
7,34.332815,0.415751,1.370423,0.005850,entropy,l2,None,"{'clf__estimator__criterion': 'entropy', 'tfid...",0.230060,0.215921,0.224066,0.223349,0.005794,7,0.799908,0.800830,0.801291,0.800676,0.000575


In [53]:
cv.best_score_

0.24179089186004815

In [54]:
cv.best_params_

{'clf__estimator__criterion': 'entropy',
 'tfidf__norm': 'l1',
 'vect__tokenizer': <function __main__.tokenize(text)>}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [59]:
Y_pred_cv = cv.best_estimator_.predict(X_test)

In [60]:
print(classification_report(Y_test,Y_pred_cv,target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4989
               request       0.83      0.46      0.59      1099
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.58      0.66      2722
          medical_help       0.59      0.08      0.15       503
      medical_products       0.74      0.07      0.13       314
     search_and_rescue       0.50      0.01      0.02       182
              security       0.00      0.00      0.00       103
              military       0.67      0.06      0.12       219
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.20      0.33       402
                  food       0.83      0.42      0.55       736
               shelter       0.80      0.34      0.48       582
              clothing       0.89      0.09      0.16        93
                 money       0.60      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [61]:
print(f1_score(Y_test,Y_pred,average='macro'))

0.238372073938


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [68]:
from sklearn.ensemble import GradientBoostingClassifier

In [69]:
list(GradientBoostingClassifier().get_params().keys())

['criterion',
 'init',
 'learning_rate',
 'loss',
 'max_depth',
 'max_features',
 'max_leaf_nodes',
 'min_impurity_decrease',
 'min_impurity_split',
 'min_samples_leaf',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'n_estimators',
 'presort',
 'random_state',
 'subsample',
 'verbose',
 'warm_start']

##### verb extractor method from class

In [78]:
from sklearn.base import BaseEstimator, TransformerMixin
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [79]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [81]:
## from Mentor:

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [74]:
from sklearn.pipeline import FeatureUnion

In [89]:
pipeline_improved = Pipeline([
    ('features', FeatureUnion([
        ('nlp_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize) ),
            ('tfidf', TfidfTransformer() )
                                  ]) ),
        ('txt_extract', StartingVerbExtractor() )
                            ]) ),      
        
    ('gbc', MultiOutputClassifier(GradientBoostingClassifier() ) )
                            ])

In [91]:
gbc = pipeline_improved.fit(X_train, Y_train)

ValueError: BinomialDeviance requires 2 classes.

In [ ]:
Y_pred_improved = pipeline_improved.predict(X_test)

In [ ]:
print(classification_report(Y_test,Y_pred_improved,target_names=Y_test.columns))

### 9. Export your model as a pickle file

In [88]:
import pickle

In [ ]:
pickle.dump(model,open(model_filepath,'wb'))

In [ ]:
import joblib

In [ ]:
joblib.dump( , "models/classifier.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.